In [7]:
from tqdm.notebook import tqdm 
import pandas as pd
from gensim.parsing.preprocessing import preprocess_string
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import (
    accuracy_score,
    classification_report
)
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB

from xgboost import XGBClassifier

from typing import List, Any, Tuple

tqdm.pandas()

In [8]:
df_train = pd.read_csv('./../../../data/train.csv')
df_val = pd.read_csv('./../../../data/val.csv')
df_test = pd.read_csv('./../../../data/test.csv')

In [9]:
df_train = pd.concat([df_train, df_val])

In [10]:
df_train.head()

,corrected_text,length,ratio_err,labels
0,the story making mona lisa smile the author ta...,423,0.037825,3
1,the last ship try land venus was three decades...,342,0.038012,2
2,electrola college not just college process the...,715,0.027972,1
3,many countries are pushing towards less cars m...,608,0.004934,5
4,voters some times get confused about the lecte...,277,0.028881,2


In [11]:
df_train['text'] = df_train['corrected_text'].progress_apply(lambda x: ' '.join(preprocess_string(x)))

  0%|          | 0/13845 [00:00<?, ?it/s]

In [12]:
print(f"vocabulary size: {len(set(' '.join(df_train['text']).split()))}")

vocabulary size: 18559


In [13]:
df_train["labels"] = LabelEncoder().fit_transform(df_train["labels"])

In [14]:
df_train.labels.value_counts()

labels
2    5024
1    3778
3    3141
0    1001
4     776
5     125
Name: count, dtype: int64

In [15]:
tfidf = TfidfVectorizer()

X_train = tfidf.fit_transform(df_train['text'])
y_train = df_train['labels']

X_train.shape

(13845, 18553)

<1x18553 sparse matrix of type '<class 'numpy.float64'>'
	with 74 stored elements in Compressed Sparse Row format>

In [19]:
def model_train(model: object,
                X: pd.DataFrame,
                y: pd.DataFrame,
                scoring: List[str])-> dict:
    """
    Return a dict of scores for the model 
    """
    scores = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'f1_macro', 'f1_weighted'], n_jobs=-1)

    return scores

In [18]:
models = {
    'RandomForestClassifier': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(),
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'XGBClassifier': XGBClassifier()
}

In [20]:
scoring = ['accuracy', 'f1_macro', 'f1_weighted', 'precision_macro', 'recall_macro']

In [21]:
scores = {}

In [25]:
for model_name, model in tqdm(models.items()):
    scores[model_name] = model_train(model, X_train.toarray(), y_train, scoring)

  0%|          | 0/5 [00:00<?, ?it/s]

In [1]:
1

1

In [2]:
scores

NameError: name 'scores' is not defined